# Program na load dat a ich analyzu

In [ ]:
# Zvacsenie buniek programu
# %matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Import kniznic

In [ ]:
# Importovanie kniznic

import py7zr
import os
import glob
import shutil
import zipfile 
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
import scipy.fftpack
from scipy.signal import spectrogram
from scipy import signal
import time
from matplotlib.colors import to_rgba
from scipy.optimize import curve_fit

# Classy na najdenie dat a ich rozbalenie

In [ ]:
# Zadefinovanie miest ktore nas zaujimaju a ciest ku nim

class Head:
    
    def __init__(self):
        disk = "Z:"
        self.places = [dl:="dl", kr:="kr", ls:="ls", ml:="ml", lfr:="lfr"]
        self.path_dl = disk+"\\dl\\hs"
        self.path_kr = disk+"\\kr\\hs"
        self.path_ls = disk+"\\ls"
        self.path_ml = disk+"\\ml\\hs"
        self.path_lfr = disk+"\\lfr\\hs"
        self.paths = {dl: self.path_dl, 
                      kr: self.path_kr,
                      ls: self.path_ls,
                      ml: self.path_ml,
                      lfr: self.path_lfr}
        
# Zadefinujeme si vstup, ake miesto a cas chceme analyzovat

class Input:
    
    def __init__(self, place, year, month, day, order):
        self.place = place
        self.year = year
        self.month = month
        self.day = day
        self.order = order

# najde vsetky png v priecinku      
def find():
    files = glob.glob("OUTPUT/*.png")
    results = []
    for i, file in enumerate(files):
    #     print(file)
        result = file[7:-4]
#         print(result)
#         print("Place ", result[0:2])
#         print("Year ", result[2:6])
#         print("Month ", result[6:8])
#         print("Day ", result[8:10])
#         print("Order ", result[10:])
#         print(30*"-")
        results.append(Input(result[0:2], result[2:6], result[6:8], result[8:10], result[10:])) 
    return results


# Nacita do velkeho dict vsetko foldre, subfoldre a archivy

def dict_all_sub(path):
    if os.path.isdir(path):
        temp = {}
        for name in os.listdir(path):
            temp[name] = dict_all_sub(os.path.join(path, name))
    else:
        temp = path
#         temp = os.path.getsize(path)
    return temp

# Overenie platnosti zadania 

def dict_all_sub_check(master, year, month, day):
    return (year in master) and (month in master[year]) and (day in master[year][month])

# Do TEMP priecinku rozbali ziadany subor

def unarchive(Master, place, year, month, day, order, unarchive_images=False):
    
    if dict_all_sub_check(Master, year, month, day):
        path = Head().paths[place]+"\\"+year+"\\"+month+"\\"+day+"\\"+order+".7z"

        with py7zr.SevenZipFile(path, 'r') as archive:
            archive.extractall(path_temp)
            
    if unarchive_images:
        path = Head().paths[place]+"\\"+year+"\\"+month+"\\"+day+"\\"+order+"_pics.zip"

        with zipfile.ZipFile(path, 'r') as archive:
            archive.extractall(path_temp)
    else: print("Nespravne zadane")
        
# loadovanie dat

class Load:
    
    def __init__(self, year, month, day, path_temp="C:\\Users\\Administrator\\bakalarka\\TEMP"):
        self.bare = list(Master[year][month][day])[0][3:-3]
        self.bin = glob.glob("TEMP\\*.bin")[0][5:] #file_bare + ".bin"
        self.txt = glob.glob("TEMP\\*.txt")[0][5:] #file_bare + ".txt"
    
        
    def signal(self):
        with open(path_temp+"\\"+self.bin, "rb") as temp:
            signal_bytes = temp.read()

        sig = np.zeros(len(signal_bytes)//2)
        for i in range(0, len(signal_bytes)//2, 1):
            sig[i] = int.from_bytes(signal_bytes[2*i:2*i+2], byteorder='little', signed=True)
        return sig
    
    def info(self):
        with open(path_temp+"\\"+self.txt, "r") as temp:
            infos=[]
            count = 0
            while True:
                count += 1
                line = temp.readline()
                if not line:
                    break
                infos.append(line.strip())

        informations = {}
        for line in infos:
            try:
                informations[line.split(":")[0].strip()] = line.split(":", 1)[1].strip()
            except: 
                informations[line.split(" ")[0].strip()] = line.split(" ", 1)[0:]

        return informations
    
    
    
        
class VecMtx():
    def __init__(self, vector, window=1024):
        
        if type(vector) != list : 
            if type(vector) != type(np.array([])): raise Exception("Not a Vector")
        self.vec = np.array(vector)
        
        self.mtx = self.vec.reshape(len(self.vec)//window, window)
        
def lin(x, a, b):
    return a*x + b

In [ ]:
class Data:
    
    def __init__(self, signal=None, info=None):
        if type(signal) == type(None): self.signal = Load().signal()
        else: self.signal = signal
            
        if type(info) == type(None): self.info = Load().info()
        else: self.info = info
        
        self.sampling = int(float(self.info["Sampling freq"]))
        self.time = np.arange(0, len(self.signal))/self.sampling
        
        self.signal = VecMtx(signal-2048)
        
        self.mean = self.signal.vec[0:10000].mean()
        self.std = self.signal.vec[0:10000].std()
        print(self.std)
        
    def fourierize(self, XXX):
        self.fourier = VecMtx(np.array([np.abs(scipy.fftpack.fft(data.signal.mtx[i+XXX]))**2 for i in range(1024*32)]).reshape(1024*(1024*32)))
        
   
    def spec_prep(self, num):
        self.fourier_mtx_mean=np.zeros((num, len(self.fourier.mtx[0])//2))
        for i in range(num):
            self.fourier_mtx_mean[i] = np.log(self.fourier.mtx[i][0:len(self.fourier.mtx[0])//2]+self.fourier.mtx[i+1][0:len(self.fourier.mtx[0])//2]+self.fourier.mtx[i+2][0:len(self.fourier.mtx[0])//2]+self.fourier.mtx[i+3][0:len(self.fourier.mtx[0])//2]+self.fourier.mtx[i+4][0:len(self.fourier.mtx[0])//2]+self.fourier.mtx[i+5][0:len(self.fourier.mtx[0])//2])/5
    
    def plot_spectrogram(self, inp, figsize=(60, 20), start=50_000, end=60_000, t=16*1024, save=False):
#         try:
#             if self.signal.vec[start*1_000//5: end*1_000//5].max()<4*self.std: return 1
#         except: return 0
        fig, axs = plt.subplots(nrows=3, ncols=1, figsize=figsize)
        plt.title(inp.place+inp.year+inp.month+inp.day+inp.order)
        
        axs[0].plot(1_000*self.time[start*1_000//5: end*1_000//5], (self.signal.vec[start*1_000//5: end*1_000//5]), color='black') 
        axs[0].set(xlabel="time (ms)", ylabel="Raw data (38 uT/s)")
        axs[0].set_xlim([start/1000, end/1000])
#         axs[0].set_ylim([-1024, 1024])
    
        
#         temp_freq = np.fft.fft(self.signal.vec[start*1_000//5: end*1_000//5])
#         temp_freq[0:30000] = 0
#         temp = np.cumsum(np.fft.ifft(temp_freq))
        
        temp = np.cumsum(self.signal.vec[start*1_000//5: end*1_000//5])
        popt, pcov = curve_fit(lin, np.arange(end*1_000//5 - start*1_000//5), np.cumsum(data.signal.vec[start*1_000//5: end*1_000//5]))
        temp = temp - np.arange(end*1_000//5 - start*1_000//5)*popt[0]
#         temp = temp-np.arange(0, len(temp))*temp[-1]/len(temp)
    
#         window = 100
#         temp = np.zeros(end*1_000//5 - start*1_000//5)    
#         for i in range(0, end*1_000//5 - start*1_000//5, window):
#             temp[i:i+window] = np.cumsum(self.signal.vec[start*1_000//5: end*1_000//5][i:i+window])
#             popt, pcov = curve_fit(lin, np.arange(end*1_000//5 - start*1_000//5)[i:i+window], np.cumsum(data.signal.vec[start*1_000//5: end*1_000//5][i:i+window]))
#             temp[i:i+window] = temp[i:i+window] - np.arange(end*1_000//5 - start*1_000//5)[i:i+window]*popt[0]
# #             temp[i:i+window] = temp[i:i+window]-np.arange(0, len(temp[i:i+window]))*temp[i:i+window][-1]/len(temp[i:i+window])
        temp = 38*5*temp/1_000_000
   
        axs[1].plot(1_000*self.time[start*1_000//5: end*1_000//5], temp, color='black') #np.cumsum
        axs[1].set(xlabel="time (ms)", ylabel="Raw data (nT)")
        axs[1].set_xlim([start/1000, end/1000])
        
        f, t, Sxx = signal.spectrogram(data.signal.vec[start*1_000//5: end*1_000//5], self.sampling, nperseg=1024, noverlap=512, mode="psd")
        Sxx[Sxx<np.percentile(Sxx, 5)]=np.percentile(Sxx, 5)
        Sxx[Sxx>np.percentile(Sxx, 95)]=np.percentile(Sxx, 95)
        cf = axs[2].pcolormesh(t, f, Sxx, shading='gouraud')
        axs[2].set_ylabel('Frequency (Hz)')
        axs[2].set_xlabel('Time (sec)')
        fig.colorbar(cf, ax=axs[2], orientation="horizontal").set_label('Intensity')
        
#         pxx,  freq, T, cax = axs[2].specgram(data.signal.vec[start*1_000//5: end*1_000//5], Fs=self.sampling, NFFT=1024, noverlap=512, mode="psd")
#         axs[2].set(xlabel="time (ms)", ylabel="Frequency (Hz)")
#         plt.colorbar(cax, orientation="horizontal").set_label('Intensity')
#         axs[2].axes.get_xaxis().set_visible(False)

        plt.tight_layout()
        if save:
            #plt.savefig("OUTPUT\\" +inp.place+inp.year+inp.month+inp.day+inp.order+"_"+str(start)+"-"+str(end)+ "__.pdf")
            np.savetxt(fname="OUTPUT\\" +inp.place+inp.year+inp.month+inp.day+inp.order+"_"+str(start)+"-"+str(end)+ "__.txt", 
                       X=self.signal.vec[start*1_000//5: end*1_000//5])
        
        plt.close()   
        return 0

# Samotne rozbalenie dat

In [ ]:
# Vytvorime si, aky chceme vstup



inp = Input("ml", "2020", "07", "03", "hs_20200703_012406")

# "Zaindexuje" si vsetky subory

if (Master_switch := True):
    Master = dict_all_sub(Head().paths[inp.place])
    print("Overenie spravnosti roku, mesiaca a dna: ", dict_all_sub_check(Master, inp.year, inp.month, inp.day))

# V domacom priecinku vytvori temporarny folder na pracu

try: shutil.rmtree(path_temp:="C:\\Users\\Administrator\\bakalarka\\TEMP")
except: print("Nepodarilo sa odstranit priecinok")
!mkdir TEMP

# try: shutil.rmtree(path_output:="C:\\Users\\Administrator\\bakalarka\\OUTPUT")
# except: print("Nepodarilo sa odstranit priecinok")
# !mkdir OUTPUT

# Do TEMP rozbali archivy

unarchive(Master, inp.place, inp.year, inp.month, inp.day, inp.order, unarchive_images=True)

# Objekt so signalom

In [ ]:
load = Load(inp.year, inp.month, inp.day)
data = Data(load.signal(), load.info())

# data.fourierize(8_192)
int(data.info["ValidPreSamples"]), int(float(data.info["Sampling freq"])), data.info["Valid samples"], 5*int(data.info["ValidPreSamples"])/1000

In [ ]:
# start=52428-40, end=52428+40

In [ ]:
data.plot_spectrogram(inp=inp, figsize=(20, 40), start=52428-500, end=52428+500, t=1024*16, save=True)

In [ ]:
arrdodatkov=[
"120530",
]

print(len(arrdodatkov))
for dodatok in arrdodatkov:
    inp = Input("ml", "2020", "08", "10", "hs_20200810_" + dodatok)

    # "Zaindexuje" si vsetky subory

    if (Master_switch := True):
        Master = dict_all_sub(Head().paths[inp.place])
        print("Overenie spravnosti roku, mesiaca a dna: ", dict_all_sub_check(Master, inp.year, inp.month, inp.day))

    # V domacom priecinku vytvori temporarny folder na pracu

    try: shutil.rmtree(path_temp:="C:\\Users\\Administrator\\bakalarka\\TEMP")
    except: print("Nepodarilo sa odstranit priecinok")
    !mkdir TEMP

    unarchive(Master, inp.place, inp.year, inp.month, inp.day, inp.order, unarchive_images=True)
    
    load = Load(inp.year, inp.month, inp.day)
    data = Data(load.signal(), load.info())
    
    data.plot_spectrogram(inp=inp, figsize=(20, 40), start=52428-500-1000, end=52428+500-1000, t=1024*16, save=True)
    
    data.plot_spectrogram(inp=inp, figsize=(20, 40), start=52428-500, end=52428+500, t=1024*16, save=True)
    
    data.plot_spectrogram(inp=inp, figsize=(20, 40), start=52428-500+2000, end=52428+500+2000, t=1024*16, save=True)
    data.plot_spectrogram(inp=inp, figsize=(20, 40), start=52428-500+3000, end=52428+500+3000, t=1024*16, save=True)
    data.plot_spectrogram(inp=inp, figsize=(20, 40), start=52428-500+4000, end=52428+500+4000, t=1024*16, save=True)
    
    data.plot_spectrogram(inp=inp, figsize=(20, 40), start=52428-500+5000, end=52428+500+5000, t=1024*16, save=True)
    data.plot_spectrogram(inp=inp, figsize=(20, 40), start=52428-500+6000, end=52428+500+6000, t=1024*16, save=True)
    data.plot_spectrogram(inp=inp, figsize=(20, 40), start=52428-500+7000, end=52428+500+7000, t=1024*16, save=True)
    data.plot_spectrogram(inp=inp, figsize=(20, 40), start=52428-500+8000, end=52428+500+8000, t=1024*16, save=True)
    data.plot_spectrogram(inp=inp, figsize=(20, 40), start=52428-500+9000, end=52428+500+9000, t=1024*16, save=True)